In [1]:
# | default_exp renderer

In [2]:
# | export

from typing import List, Dict, Any, Optional
from fasthtml.common import *
from pylogue.cards import ChatCard
from pylogue.session import Message, ChatSession

## Presentation Layer

This module provides rendering components for the chat interface.

In [ ]:
# | export


class ChatRenderer:
    """Renders chat components with customizable styling."""

    CHAT_DIV_ID = "chat-cards"

    def __init__(
        self,
        card: Optional[ChatCard] = None,
        input_placeholder: str = "Type a message...",
        input_style: Optional[str] = None,
        chat_container_style: Optional[str] = None,
        ws_endpoint: str = "/ws",
    ):
        """
        Initialize ChatRenderer.

        Args:
            card: ChatCard instance for rendering messages
            input_placeholder: Placeholder text for input field
            input_style: Custom CSS style for input field
            chat_container_style: Custom CSS style for chat container
            ws_endpoint: WebSocket endpoint path
        """
        self.card = card or ChatCard()
        self.input_placeholder = input_placeholder
        self.ws_endpoint = ws_endpoint
        self.input_style = input_style or (
            "width: 60%; max-width: 600px; padding: 0.75em; "
            "font-size: 1em; border-radius: 0.5em"
        )
        self.chat_container_style = chat_container_style or (
            "display: flex; flex-direction: column; gap: 10px;"
        )

    def render_message(self, message: Message) -> Any:
        """Render a single message."""
        return self.card(message.to_dict())

    def render_messages(self, messages: List[Message]) -> Any:
        """
        Render a list of messages in a container.

        Args:
            messages: List of Message objects to render

        Returns:
            FastHTML Div containing all rendered messages
        """
        return Div(
            *[self.render_message(msg) for msg in messages],
            id=self.CHAT_DIV_ID,
            cls="chat-cards",
            style=self.chat_container_style,
        )

    def render_messages_from_dicts(self, message_dicts: List[Dict[str, Any]]) -> Any:
        """
        Render messages from dictionary representations.

        Args:
            message_dicts: List of message dictionaries

        Returns:
            FastHTML Div containing all rendered messages
        """
        messages = [Message.from_dict(d) for d in message_dicts]
        return self.render_messages(messages)

    def render_input(self, input_id: str = "msg", autofocus: bool = True) -> Any:
        """
        Render the message input field.

        Args:
            input_id: HTML ID for the input element
            autofocus: Whether to autofocus the input

        Returns:
            FastHTML Input element
        """
        return Input(
            id=input_id,
            placeholder=self.input_placeholder,
            autofocus=autofocus,
            style=self.input_style,
        )

    def render_form(
        self,
        form_id: str = "form",
        form_style: Optional[str] = None,
        ws_send: bool = True,
    ) -> Any:
        """
        Render the input form with WebSocket connection.

        Args:
            form_id: HTML ID for the form
            form_style: Custom CSS style for form
            ws_send: Whether form sends via WebSocket

        Returns:
            FastHTML Form element
        """
        form_style = form_style or (
            "display: flex; justify-content: center; "
            "margin-top: 20px; padding: 20px;"
        )

        return Form(
            self.render_input(),
            id=form_id,
            hx_ext="ws",
            ws_connect=self.ws_endpoint,
            ws_send=ws_send,
            style=form_style,
        )

## Test the Renderer

In [4]:
from fasthtml.jupyter import render_ft

render_ft()

In [5]:
# Test basic rendering
from pylogue.session import Message

renderer = ChatRenderer()
msg = Message(role="User", content="Hello!")
renderer.render_message(msg)

<div>
  <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>    <div class="marked" style="white-space: pre-wrap;">Hello!</div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [6]:
# Test message list rendering
messages = [
    Message(role="User", content="Hi there!"),
    Message(role="Assistant", content="Hello! How can I help?"),
    Message(role="User", content="What's the weather?"),
]

renderer.render_messages(messages)

<div>
  <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
    <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hi there!</div>
    </div>
    <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🕵️‍♂️ Assistant: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hello! How can I help?</div>
    </div>
    <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>      <div class="marked" style="white-space: pre-wrap;">What's the weather?</div>
    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [7]:
# Test input rendering
renderer.render_input()

<div>
  <input placeholder="Type a message..." autofocus id="msg" style="width: 60%; max-width: 600px; padding: 0.75em; font-size: 1em; border-radius: 0.5em" name="msg">
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [8]:
# Test full form
renderer.render_form()

<div>
<form enctype="multipart/form-data" ws-send id="form" style="display: flex; justify-content: center; margin-top: 20px; padding: 20px;" name="form">    <input placeholder="Type a message..." autofocus id="msg" style="width: 60%; max-width: 600px; padding: 0.75em; font-size: 1em; border-radius: 0.5em" name="msg">
</form><script>if (window.htmx) htmx.process(document.body)</script></div>


In [9]:
# Test complete interface
renderer.render_chat_interface(messages=messages, title="My Custom Chat")

<div>
  <div>
    <h1 style="text-align: center; padding: 1em;">My Custom Chat</h1>
    <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
      <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>        <div class="marked" style="white-space: pre-wrap;">Hi there!</div>
      </div>
      <div style="background: #3B3B3B; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🕵️‍♂️ Assistant: </u></span>        <div class="marked" style="white-space: pre-wrap;">Hello! How can I help?</div>
      </div>
      <div style="background: #272727; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🗣️ User: </u></span>        <div class="marked" style="white-space: pre-wrap;">What's the weather?</div>
      </div>
    </div>
<form enctype="multipart/form-data" ws-send id="form" style="display: flex; justify-content: center; margin-top: 20px; padding: 20px;" name="form">      <input placeholder="Type a message..." autofocus id="msg" style="width: 60%; max-width: 600px; padding: 0.75em; font-size: 1em; border-radius: 0.5em" name="msg">
</form>  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>


In [11]:
# Test with custom card styling
custom_card = ChatCard(
    user_color="#482F00",
    assistant_color="#005460",
    user_emoji="👤",
    assistant_emoji="🤖",
)

custom_renderer = ChatRenderer(card=custom_card)
custom_renderer.render_messages(messages)

<div>
  <div id="chat-cards" class="chat-cards" style="display: flex; flex-direction: column; gap: 10px;">
    <div style="background: #482F00; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>👤 User: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hi there!</div>
    </div>
    <div style="background: #005460; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: left; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>🤖 Assistant: </u></span>      <div class="marked" style="white-space: pre-wrap;">Hello! How can I help?</div>
    </div>
    <div style="background: #482F00; padding: 10px; font-size: 1.5em; width: 60%; align-self: center; text-align: right; border-radius: 1em; padding: 1.25em">
<span style="font-weight: bold; font-size: 1.1em; display: block; margin-bottom: 8px;"><u>👤 User: </u></span>      <div class="marked" style="white-space: pre-wrap;">What's the weather?</div>
    </div>
  </div>
<script>if (window.htmx) htmx.process(document.body)</script></div>
